# 02. Chat Only

Attempt to use only the chat data to answer questions

> As we can see, the concept is not bad, but we cannot send the entire chat history to OpenAI, because it has a limit of 4096 tokens. So we need to find a way to summarize the chat history or pass just relevant text.

In [39]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv

from IPython.core.display import Markdown

## Configure Environment

reference: [Using .env Files for Environment Variables in Python Applications](https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1)

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_CHAT_MODEL = os.getenv('OPENAI_CHAT_MODEL', 'gpt-3.5-turbo')

openai.api_key = OPENAI_API_KEY

## Load Data

In [3]:
df_source = pd.read_csv('data/output/transcripts.csv')

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

(140, 5)


,course,topic,title,url,transcript
0,DSM100-2022-OCT,Module information,Module introduction video,https://learn.london.ac.uk/mod/page/view.php?i...,-Welcome to AI Module. Artificial intelligence...
1,DSM100-2022-OCT,Module information,Meet the team,https://learn.london.ac.uk/mod/page/view.php?i...,[music]-Welcome to the AI module. My name is L...
2,DSM100-2022-OCT,Topic 1: Introduction,Lecture: Introduction to Topic 1,https://learn.london.ac.uk/mod/page/view.php?i...,"Welcome to topic one, Introduction to AI. In t..."


## Functions

In [4]:
def create_session_history(data: pd.DataFrame) -> dict:
    session_history = []

    for index, row in data.iterrows():
        session_history.append({
            'role': 'user',
            'content': row['transcript']
        })

    return session_history

#create_session_history(df_source)

In [7]:
def ask_question(question: str, session_history: list) -> str:
    messages = session_history.copy()

    # add the user message
    messages.append({
        'role': 'user',
        'content': question
    })

    response = openai.ChatCompletion.create(
        model=OPENAI_CHAT_MODEL,
        messages=messages
    )

    return response['choices'][0]['message']['content']

# test the function
question = 'Reinforcement learning might be described as learning a Q function. If we are using Q-learning to learn how to play video games, state TWO aspects that make Q-learning difficult.'
#ask_question(question, create_session_history(df_source))

## Summarize

In [24]:
ask = f'Summarize the following: {df_source.iloc[101].transcript}'
summary = ask_question(ask, [])

print(summary)

The mini lecture discusses how abduction can be used to generate hypotheses, and how robot scientist Adam used it to formulate new hypotheses in the field of functional genomics. Deductive reasoning is contrasted with abductive reasoning, and it is noted that our knowledge may be incomplete and inaccurate, requiring further testing and experimentation to update it. The metabolic pathways model for yeast is used to illustrate how Adam identified possible candidate genes for controlling the biosynthesis of phenylalanine, tyrosine, and tryptophan, and how he used bio informatics databases to narrow down the potential options. The lecture emphasizes that hypotheses generated through abduction are plausible but must be experimentally validated before being added to the metabolic pathways model.


In [59]:
transcript = df_source.iloc[43].transcript
word_limit = 1000

#ask = f'Keep as much detail as possible in {word_limit} words: {transcript}'
#ask = f'could you give me a summary of this text in around {word_limit} words? {transcript}'
#ask = f'Produce a list of bullet points of the knowledge in this text: {transcript}'
ask = f'provide each concept explained as a bullet list: {transcript}'

summary = ask_question(ask, [])

# print the transcript and the summary length
print('Transcript length:', len(transcript))
print('Summary length:', len(summary))

# print the transcript
print('--- Transcript ---')
display(Markdown(transcript))

# print the summary
print('--- Summary ---')
display(Markdown(summary))

Transcript length: 16822
Summary length: 1596
--- Transcript ---


[MUSIC][MUSIC]-In this video, I'm going to explain how genetic algorithms work. In summary, I'm going to start out by just explaining, giving you a brief definition of genetic algorithms. Then we're going to talk about encoding with a genotype, decoding and testing with a phenotype. Then we'll talk about selection and breeding and mutation and go back tothe start again to show that it's an iterated process. Let's start out by looking at some definitions. The question is who invented genetic algorithms?John Holland, from the University of Michigan, is credited with a lot of the early work in the '60s and '70s on genetic algorithms. Obviously, many other researchers contributed as well. Since then, many more have come in and added all kinds of things to it. John Holland's 1975 book Adaptation in Natural and Artificial System sis considered to be the seminal textbook if you like, earliest textbook in this area. What about a definition of genetic algorithms?It's a probabilistic search procedure designed to work on large spaces involving states that can be represented as strings. Who said that?It is Goldberg and Holland. Holland, we just mentioned. Goldberg's another key researcher in the field. Let's pick that apart a bit. It's a probabilistic search procedure. Probabilistic search procedure. That means that it's essentially--a genetic algorithm is a search algorithm. That's one thing to know. The question is, what's it searching for?Well, it's searching through a large space and the space contains states. In that space, there are a whole bunch of different states, and those states are described or represented using strings. That's the abstract description of what a genetic algorithm isand what it does. Now, the question is, how does that relate to evolution theory, if you like, why is it called a genetic algorithm?Well, we've got selection going on up there. The probabilistic search is this process of selection and breeding. Then down there, we've got the representation, which is the genotype. The string representation is the genotype. Over there, way over there, we've got the phenotype, which is the states if you like. Typically, in a genetic algorithm, it's simplified a bit. It's simplified into-- the state is directly derived from the string, the genotype. The phenotype is directly derived from the genotype, typically. Unlike natural evolution, where the genotype gets expressed, and then evaluated in natural world. It's a simplification of natural evolution and abstraction of itto extract the useful behaviors and characteristics of natural evolution. What's the point of all this?That's all very abstract. What's the point of it all?We define the space. The space that we're searching through is the space of possible solutions to a problem. A genetic algorithm is a search algorithm that's searching for solutions to a problem, which is normally what a search algorithm should be doing. When you do your search on your search engine you're--The problem is, I don't know which website to look at for this particular thing. It searches and it sends you back, "Here's a set of possible websites. "Similarly, instead of it being a search for websites, we're searching for solutions to problems. The space or the states are the solutions and the strings are how we describe those solutions. Typical examples of problems are listed below. You can see we have, for example, what's the ideal shape for an aero plane wing?That's the example we're going to be using in this video. Another one might be what are the ideal settings for my data center to reduce power consumption?Then the final one is the final example, what's the most efficient robot form for walking?Shocker, that's what we're going to be doing. In later videos, we're going to be actually building a genetic algorithm which allows us to evolve robots which can move, basically. Our space of possible problem is, we don't know how to design a robot that can move, and so the space of possible solutions will bea whole bunch of different shapes and configurations, which allow us to find one. Moving on. What I want to do now is jump over to my editor program over here. What we're going to do now is we're going to look at how we can possibly encode a problem as a string and then we're going to work through and see how we can then manipulate that string to implement the genetic algorithm. Let's first off, think about a problem. I'm going to take the aero plane wing problem. Imagine I need to know what is the best aero plane wing design?I've got a range of different designs. I've got a flat wing. I've got a triangular one. Maybe I've got a curvy one like that. I could even have something that looks more like that. I've got four different shapes of wing. The question is, how might I describe those using some string representation?Remember, the string can have anything. Any characters in, whatever you want. Commonly in genetic algorithms, the stringis actually a binary string, zeros, and ones. I want you to pause the video and have a think about how you would describe these various wing shapes using a standardized string description. Maybe pause the video and have a think about it. I'm back again, if you paused it. What we're going to do is now come up with a solution to this. The first thing is, I noticed that there's four distinct shapes here. Maybe I can start out by saying, "I need some bits to describe which shape it is. If there's four, I can use two bits to describe that. "I could say, maybe this one's going to be represented by two zeros. This one's going to be a zero and a one. That's going to be a one and a zero. That's going to be a one and a one, so with two bits, I've now described each of the possible shapes. The first two bits of my representation are, in fact, going to be the shape of the wing there. Next up, I might say, "Well, what other characteristics does this wing have?"Well, I could say, "It's got a length like that. "Each of them has a distinct length. That might be the next two bits or three bits. Maybe I could be generous and say, "I can have eight different lengths. "I'm going to have three bits for that so that's my shape. Then I have another bit, which is another three bits for the length maybe. I can have eight different lengths. Then the next thing is, I might say, "It's got this idea of being bent around. "It's got this kind of angle concept. I might say, "Let's have a look at the angle. " That one has got this angle like that. Then this one has that angle. This one's got that angle, and so on. I define an angle for each one if it's bent or not. That allows-- I could do that with another. Maybe I can be generous with the angle and maybe have four bits or whatever. It's up to you to describe how precisely each of these things is represented. Now, I've actually represented a wing as a bit string. That's great. I've now got my-- The space of all possible bit strings. Think about that, it's 2^9. What's that?512?I've got 512 possible wings that I could have with that bit string. That's pretty good. That's a good starting point for my genetic algorithm. Generally, you'd have a longer bit string to give you a really large space. Remember that definition, where it said a large space 512points is not really a large space, is it?Imagine a much bigger space. Anyway, that's just a simple example. I've got my genotype. That's my genotype. The next step is to express that genotype into a phenotype, which remember, is the physical expression of this. We know how we're going to do that. We're basically going to chunk through those bits and interpret it into our wing and say, "Well, okay, that's going to be a wing. "The question is, how do I then?Now, I've got that description of a wing, how do I then test that and see how good it is?That's the next step. I'm just going to pause again. I want you to have a quick think, how am I going to test this wing?How would you test these different wings in a computer?I'm going to show you how I'd think about doing it. I might say, "I'm going to build myself a really basic physics simulation, where I basically get my two-dimensional wing shape. "It's going to be 2D physics, really easy, and I'm going to blow some wind at it I'm going to create some forces. I'm going to blow those at the wing. I'm going to simply measure what the force is going up. If you like, the uplift. What's the lift generated by that wing?My fitness, if you like, the result ofmy evaluation is how big that is there. How much does it raise up?We literally have a physics simulation. I'd instantiate each wing into a little object in the physics world and I would run it with some wind, and I would measure how far it goes up. That would give me a fitness score fo reach of the wings in my initial population. It's worth saying that you need a starter population. I'll start out with a population of a whole bunch of variants of the wings. Maybe one will be like that, maybe like that, and maybe a triangle, maybe a bent triangle, and so on. Maybe I'd start with, say 10, but typically in a real genetic algorithm, you'd have thousand sin your initial population, possibly. It depends how computationally expensive your physics simulation is, and so on. There's various things. Yes, it's fine to say, we've got a population of different things, we're going to test them all, and say, "Okay, what's the lift on each of them when we blow wind at it?"Then that gives us a score for everything in the population. That's the first part of the testing and the breeding. The next step is, I need to say, "Okay, I need to generate the next generation, "because it's all very well to have scored maybe 10 solutions. I could just stop here and say, "Well, I'll just get the best one out of the 10. "If you've randomly generated them, you can't guarantee you're going to have a good solution so you need to be able to iterate on these. That's the whole point of the genetic algorithm, you're searching, you don't just do a one-step search and that's your solution. Actually, it's an iterative search through this space. That's what we need to do. How do we iterate?Well, so the next step is to select two of the individuals for breeding. Okay, how does breeding work?How does selection work?Well, let's do selection first. Selection works like this. It's called roulette wheel selection. I'll draw a wheel or a wheel of fortune selection but yes, typically in the literature, it's called roulette wheel selection. Let's say I've got a population of four to make it simple for me to draw. Okay, so I've got four, that would give me four segments. One, two, three, four. I've got four segments, and the angle of the segment, that angle, or that angle, rather, that angle is basically the fitness. It really fits wings, so maybe the triangle wing does really well in the simulation, maybe a weird stubby wing like that doesn't do so well. Maybe the banana wing is the worst one. Maybe a really thin triangle. Say I had four individuals, I'd measure the fitness of each, and the higher the fitness, the greater the angle it gets on the wheel of fortune, the roulette wheel. The next step is, so if you've ever watched a James Bond film, or whatever, or been to a casino, et cetera, you spin it around, so you spin around the roulette wheel, and wherever it stops, that's the first parent. Then you spin it around again and that'll give you the second parent. Now, you can imagine that the more of this you have, the higher the chance of you being selected when you spin around. That means that the fittest individuals are more likely to get selected. It also means that the less fit ones have a possibility of being selected as well so they don't get completely discarded, just in case they've got some useful stuff in there. I'd end up with two individuals. You can see I've selected the triangle one, and the stubby square wing. Though that's my selection process, and I would do that multiple times, to get multiple pairs of parents. The next step is I need to take those two parents, and I need to recombine them in some way to generate the next generation. The first individual that's going into the next generation is going to be some combination of those two. How am I going to do it?Well, there's two steps, there's crossover and then there's mutation. Let's look at those two and see how they work. This is crossover. Imagine I've got my two genomes or genotypes, like this. That would be the triangle, and that would be the other one I selected. Those are the two parents I've selected. Crossover essentially is looking at taking part of one and then part of the other. One point crossover would be-- I'm going to take the top, maybe that bit of that genome and that bit of that genome and that would give me the next generation. You can also do a multi point crossover. You can say, "Well, I'm going to get that bit and that bit, "or with really long genomes, maybe youcan end up with different whatever you want All kinds of combinations, but the basic one is say, we'll call it single-point crossover like that. You would typically use a random number to get to choose where you crossover or maybe you'd look at the fitness and say, "Well, again, I'm going to apply the fitness again and allow it to choose so the fitter individual, I'll take more of its genome. "Again, you've already applied the selection pressure, so why reapply it?I think in the previous video, I've given you a link to this Inman Harvey paper where he goes through and really picks apart the genetic algorithm says, "Well, why would you apply selection pressure twice?You've got selection pressures, just apply it once and simplify your algorithm. "Let's just say it's a random number. What's the result of that?Well, if I just change the color for the bottom one, I'll end up with that of that one, and that of that one, and that's my new genotype. That is my new genotype, it's got abit of one parent, a bit of the other. If I was to express that into a phenotype, maybe I'd end up with, I don't know a slightly, slightly blockier triangle, I don't know, whatever it is. The bottom one is a combination of the other two. That's how I do selection and breeding. Remember, we need to do mutation now. The final step is to take this child and then we mutate it slightly. Again, randomly, we would choose places on the genome to mutate it. If we just redraw it and put in those mutations, what I'm going to do is I'm going to pick some random points on it, I'll say, well, that bit there that bit is going to flip, that bit's going to flip. Depending on, you have some percentage of them that get mutated. You could basically flip the bits on those, and then you end up with a slight variation. You've got a combination of the two parents with a bit of change, to give you a nice amount of variation. Remember, we do this multiple times, we go back, we do the roulette wheel, we spin it around until we get lots of pairs of parents, and we do the crossover mutation. Eventually, we end up with, say 10, or whatever our population size is, we end up with a whole bunch of new genomes in our population. Then, of course, we go back to the start again, we go right back to this step We evaluate the whole population again and then we do the selection. We do the crossover and mutation and we keep going, and we go around and around until eventually, we end up witha population of really good solutions. As I'll show you in a later video, so John Holland's schema theorem describes why this works. It explains why doing this weird looking process, why it allows you to search highly complicated spaces, and find decent solutions in those spaces that other types of algorithms might not be able to find. For example, hill climbing, where you just always choose the next, you mutate and find the best next one, you don't have a population doing crossover and all of that We put all these features into the genetic algorithm. Holland describes why they work using his schema theorem, which we'll talk a bit about in a later video. Okay, I think that's it. Let's just summarize. We've just been talking about the genetic algorithm. I've told you who's credited with the main seminal work on this, which is Holland. Then we've talked about encoding. How we take a problem and express it in the form of a string of bits in our case, or numbers. Then we talked about how we express that and test it, how we convert the string of bits, and run it and test it, and that gives us scores for various strings. Then I talked about how we select and breed those strings to produce the next generation and how that continues iteratively until we havea whole population that's fit. Okay, in this video, we've just seen a worked example of how genetic algorithms work.

--- Summary ---




- Genetic algorithms are a probabilistic search algorithm designed to work on large spaces involving states that can be represented as strings.
- John Holland from the University of Michigan is credited with the early work in the 60s and 70s on genetic algorithms, and his book "Adaptation in Natural and Artificial System" is considered seminal.
- The strings used in genetic algorithms are called genotypes, and they are represented in binary form.
- The states or solutions in the space are the phenotypes and are derived directly from the genotype.
- The goal of a genetic algorithm is to search for solutions to a problem defined in the space of possible solutions.
- Typical problems that can be solved with genetic algorithms include finding the ideal shape for an aeroplane wing, ideal data center settings to reduce power consumption, and the most efficient robot form for walking.
- Encoding involves expressing the problem using a standardized string representation, such as a binary string.
- Testing involves expressing the genotype as a phenotype in a simulation and evaluating it based on defined fitness criteria.
- Selection involves choosing individuals with the highest fitness to breed in the next generation.
- Breeding involves crossover, where parts of the parent's genotypes are combined to create a new genotype, and mutation, where random changes are made to the new genotype.
- The genetic algorithm process is repeated iteratively until a fittest population of solutions is found.
- John Holland's schema theorem explains why genetic algorithms work in searching for

## Search Keywords

In [25]:
# suggest search keywords
ask = f'suggest search keywords for the following as a comma seperated list: {question}'
response = ask_question(ask, [])

keywords = response.replace('\n', '').split(',')
print(keywords)

['Reinforcement learning', ' Q function', ' Q-learning', ' video games', ' difficulty', ' aspects', ' learning.']
